In [58]:
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re
import gc
from sklearn import preprocessing
import matplotlib as mpl
from functools import wraps
import os
from sklearn.preprocessing import LabelEncoder
# from sklearn.externals import joblib
from joblib import Parallel, delayed
import multiprocessing
from datetime import timedelta
from io import StringIO
# from sklearn.externals import joblib
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from random import random

pd.set_option('display.max_columns',None) 
pd.set_option('expand_frame_repr', False)

#我的多核函数，可以提高训练速度，考虑到总行复现环境不确定性，就没有用
def applyParallel(dfGrouped, func):
    ret = Parallel(n_jobs=multiprocessing.cpu_count()-6)(delayed(func)(name,group) for name, group in dfGrouped)
    return pd.concat(ret)

#调参类定义，传入多个参数组成的一个字典，返回一个迭代器，感觉比sklearn中的调参函数好用，个人习惯吧
class params_iter:
    def getPlans(self,lis,jude=True):
        if jude: 
            lis = [[[i] for i in lis[0]]] + lis[1:]
        if len(lis) > 2:
            for i in lis[0]:
                for j in lis[1]:
                    self.getPlans([[i + [j]]] + lis[2:], False)
        elif len(lis) == 2:
            for i in lis[0]:
                for j in lis[1]:
                    self.param_list.append(i + [j])
                
    def __init__(self,params):
        self.params=params
        self.cur_index=0
        self.param_list=[]
        val=list(params.values())
        keys=list(params.keys())
        self.getPlans(val)
        self.df=pd.DataFrame(param_list,columns=keys)
        
    def __iter__(self):
        return self
 
    def __next__(self):
        x = self.df.iloc[self.cur_index,:]
        self.cur_index += 1
        return x.to_dict()
    
def get_sample(dataset,target_0=0.33,target_1=0.33,seed=None):
    dataset=dataset.copy()
    x0=dataset[dataset['target']==0]
    x1=dataset[dataset['target']==1]
    x0_selected=x0.sample(frac=target_0,random_state=seed)
    x1_selected=x1.sample(frac=target_1,random_state=seed)
    dataset=pd.concat([x0_selected,x1_selected]).sample(frac=1,random_state=seed)
    y_selected=dataset['target']
    x_selected=dataset.drop(columns =['target'])
    return x_selected,y_selected

dataset_train=pd.read_hdf('dataset/1_1_特征衍生.h5', key='TRAIN')
dataset_test=pd.read_hdf('dataset/1_1_特征衍生.h5', key='PREDICT')



In [52]:

models=[]
feature=['crcd_is_overdue', 'cus_sex', 'crcd_is_pc', 'crcd_is_sup', 'cus_type', 'crcd_is_gec', 'crcd_bachange_cnt', 'cus_intvtime_trans',
         'cus_edu', 'cus_marr_stat', 'cus_occu', 'cus_oact_pla', 'cus_os_dist', 'crcd_is_gc', 'crcd_bill_mcnt',
                 
        'cus_age', 'cus_mbank_lcnt', 'cus_active_ind', 'pass_date', 'interval_mon_count', 'interval_date_1stc', 'interval_date_1stc12', 'cus_hisloan_cnt', 'crcd_mavg_balind', 'cus_region_move_ind', 'cus_reloan_cap_ind', 'cus_asset_bal', 'crcd_max_quota', 'crcd_lmon_due_amount', 'crcd_due_amount', 'crcd_mavg_balance', 'crcd_points', 'a_mmax_tranbal', 'a_mavg_tranbal', 'b_mmax_tranbal', 'b_mavg_tranbal', 'c_mmax_tranbal', 'c_mavg_tranbal', 'd_mmax_tranbal', 'd_mavg_tranbal', 'e_mmax_tranbal', 'e_mavg_tranbal', 'f_mmax_tranbal', 'f_mavg_tranbal', 'g_mmax_tranbal', 'g_mavg_tranbal', 'h_mmax_tranbal', 'h_mavg_tranbal', 'i_mmax_tranbal', 'i_mavg_tranbal', 'j_mmax_tranbal', 'j_mavg_tranbal', 'k_mmax_tranbal', 'k_mavg_tranbal', 'm_mmax_tranbal', 'm_mavg_tranbal', 'n_mavg_trancount', 'n_mavg_tranbal', 'a_mdivision_tranbal', 'b_mdivision_tranbal', 'c_mdivision_tranbal', 'd_mdivision_tranbal', 'e_mdivision_tranbal', 'f_mdivision_tranbal', 'g_mdivision_tranbal', 'h_mdivision_tranbal', 'i_mdivision_tranbal', 'j_mdivision_tranbal', 'k_mdivision_tranbal', 'm_mdivision_tranbal', 'n_mdivision_tranbal', 'crcd_due_div_crcd_max_quota', 'crcd_due_div_crcd_lmon_due', 'a_div_cus_asset_bal', 'b_div_cus_asset_bal', 'c_div_cus_asset_bal', 'd_div_cus_asset_bal', 'e_div_cus_asset_bal', 'f_div_cus_asset_bal', 'g_div_cus_asset_bal', 'h_div_cus_asset_bal', 'i_div_cus_asset_bal', 'j_div_cus_asset_bal', 'k_div_cus_asset_bal', 'm_div_cus_asset_bal', 'n_div_cus_asset_bal'
        ]
# feature=['crcd_is_overdue', 'cus_sex', 'crcd_is_pc', 'crcd_is_sup', 'cus_type', 'crcd_is_gec', 'crcd_bachange_cnt',
#              'cus_intvtime_trans', 'cus_edu', 'cus_marr_stat', 'cus_oact_pla', 'crcd_is_gc', 'crcd_bill_mcnt',
#              'cus_age', 'cus_mbank_lcnt', 'cus_active_ind', 'pass_date', 'interval_mon_count', 'interval_date_1stc', 'interval_date_1stc12', 'cus_hisloan_cnt', 'crcd_mavg_balind', 'cus_region_move_ind', 'cus_reloan_cap_ind', 'cus_asset_bal', 'crcd_max_quota', 'crcd_lmon_due_amount', 'crcd_due_amount', 'crcd_mavg_balance', 'crcd_points', 'a_mmax_tranbal', 'a_mavg_tranbal', 'b_mmax_tranbal', 'b_mavg_tranbal', 'c_mmax_tranbal', 'c_mavg_tranbal', 'd_mmax_tranbal', 'd_mavg_tranbal', 'e_mmax_tranbal', 'e_mavg_tranbal', 'f_mmax_tranbal', 'g_mmax_tranbal', 'g_mavg_tranbal', 'h_mmax_tranbal', 'h_mavg_tranbal', 'i_mmax_tranbal', 'i_mavg_tranbal', 'j_mmax_tranbal', 'j_mavg_tranbal', 'k_mmax_tranbal', 'k_mavg_tranbal', 'm_mmax_tranbal', 'm_mavg_tranbal', 'n_mavg_trancount', 'n_mavg_tranbal', 'a_mdivision_tranbal', 'b_mdivision_tranbal', 'd_mdivision_tranbal', 'e_mdivision_tranbal', 'g_mdivision_tranbal', 'h_mdivision_tranbal', 'i_mdivision_tranbal', 'j_mdivision_tranbal', 'k_mdivision_tranbal', 'm_mdivision_tranbal', 'n_mdivision_tranbal', 'crcd_due_div_crcd_lmon_due', 'a_div_cus_asset_bal', 'b_div_cus_asset_bal', 'c_div_cus_asset_bal', 'd_div_cus_asset_bal', 'e_div_cus_asset_bal', 'f_div_cus_asset_bal', 'g_div_cus_asset_bal', 'h_div_cus_asset_bal', 'i_div_cus_asset_bal', 'j_div_cus_asset_bal', 'k_div_cus_asset_bal', 'm_div_cus_asset_bal', 'n_div_cus_asset_bal']

cago_feature='0,1,2,3,4,5,6,7,8,9,10,11,12,13,14'
feature_train=feature+['target']
feature_test=feature
dataset_train=dataset_train[feature_train]
dataset_test=dataset_test[feature_test]

def run(thread):
    params={'reg_alpha': 1, 'reg_lambda': 0.8, 'subsample': 1, 'colsample_bytree': 1,
        'max_depth': 4, 'num_leaves': 18, 'min_data_in_leaf': 20, 
        'bagging_freq': 6, 'bagging_fraction': 0.65,
        'feature_fraction': 0.55, 'min_gain_to_split': 0.2, 
        'min_child_weight': 19.428902804238373, 'objective': 'binary',
        'learning_rate': 0.01,'boosting': 'gbdt', 'verbosity': -1, 'metric': 'auc', 
        'num_threads': thread,
        'is_unbalance': True, 'categorical_feature': cago_feature}
    
    dataset_train_x,dataset_train_y=get_sample(dataset_train,target_0=1,target_1=1,seed=None)
    X_train,X_test,Y_train,Y_test=train_test_split(dataset_train_x[feature],dataset_train_y,test_size=0.2,random_state=None)



    lgb_train = lgb.Dataset(X_train,Y_train)
    lgb_eval = lgb.Dataset(X_test,Y_test,reference=lgb_train)

    model=lgb.train(params,lgb_train,num_boost_round=4000,valid_sets=lgb_eval,early_stopping_rounds=200)#,init_model=model)
    return model

thread=2
models=Parallel(n_jobs=int(42/thread))(delayed(run)(thread) for fea in range(int(42/thread)))
print('end')


In [53]:
dataset_train_x,dataset_train_y=get_sample(dataset_train[feature_train],target_0=1,target_1=1,seed=None)

def run(i,model):
    y_pr=model.predict(dataset_train_x)
    return y_pr
y_pres=Parallel(n_jobs=int(42/thread))(delayed(run)(i,model) for i,model in enumerate(models))
y_pres=pd.DataFrame(np.array(y_pres).T)
# for i,model in enumerate(models):
#     y_pres[i]=model.predict(dataset_train_x)
y_pr=y_pres.mean(axis=1)
y_pr=y_pr.astype(np.float16)
y_pr.index=dataset_train_x.index
offline_test_auc=roc_auc_score(dataset_train_y,y_pr)
print('线下测试集AUC:%f'%offline_test_auc)

线下测试集AUC:0.802173


In [54]:
def run(i,model):
    y_pr=model.predict(dataset_test)
    return y_pr
y_pres=Parallel(n_jobs=int(42/thread))(delayed(run)(i,model) for i,model in enumerate(models))
y_pres=pd.DataFrame(np.array(y_pres).T)

y_pr=y_pres.mean(axis=1)
y_pr.index=dataset_test.index
y_pr.to_csv('online/LOAN_JS_FH_SX_04_20191203_03.csv',header=['target'])

In [55]:
feature_importance=pd.Series(model.feature_importance(),index=model.feature_name())
feature_importance.sort_values(ascending=False)

cus_oact_pla            2075
crcd_bill_mcnt          1101
pass_date                719
crcd_points              594
crcd_mavg_balance        515
interval_mon_count       419
cus_reloan_cap_ind       399
g_mdivision_tranbal      391
cus_intvtime_trans       358
m_mavg_tranbal           357
interval_date_1stc       334
cus_age                  313
cus_sex                  307
a_mavg_tranbal           305
crcd_due_amount          298
a_mmax_tranbal           294
cus_mbank_lcnt           265
m_mdivision_tranbal      244
crcd_is_overdue          237
interval_date_1stc12     235
cus_asset_bal            226
crcd_max_quota           218
cus_active_ind           217
d_mavg_tranbal           211
crcd_bachange_cnt        207
b_mmax_tranbal           195
cus_edu                  191
a_mdivision_tranbal      186
crcd_mavg_balind         184
crcd_lmon_due_amount     182
                        ... 
e_mdivision_tranbal      105
e_mavg_tranbal           105
i_mmax_tranbal           103
c_mmax_tranbal

# 测试

In [67]:
dataset_train=pd.read_hdf('dataset/1_1_特征衍生.h5', key='TRAIN')
dataset_test=pd.read_hdf('dataset/1_1_特征衍生.h5', key='PREDICT')

feature=['crcd_is_overdue', 'cus_sex', 'crcd_is_pc', 'crcd_is_sup', 'cus_type', 'crcd_is_gec', 'crcd_bachange_cnt', 'cus_intvtime_trans',
         'cus_edu', 'cus_marr_stat', 'cus_occu', 'cus_oact_pla', 'cus_os_dist', 'crcd_is_gc', 'crcd_bill_mcnt',
                 
        'cus_age', 'cus_mbank_lcnt', 'cus_active_ind', 'pass_date', 'interval_mon_count', 'interval_date_1stc', 'interval_date_1stc12', 'cus_hisloan_cnt', 'crcd_mavg_balind', 'cus_region_move_ind', 'cus_reloan_cap_ind', 'cus_asset_bal', 'crcd_max_quota', 'crcd_lmon_due_amount', 'crcd_due_amount', 'crcd_mavg_balance', 'crcd_points', 'a_mmax_tranbal', 'a_mavg_tranbal', 'b_mmax_tranbal', 'b_mavg_tranbal', 'c_mmax_tranbal', 'c_mavg_tranbal', 'd_mmax_tranbal', 'd_mavg_tranbal', 'e_mmax_tranbal', 'e_mavg_tranbal', 'f_mmax_tranbal', 'f_mavg_tranbal', 'g_mmax_tranbal', 'g_mavg_tranbal', 'h_mmax_tranbal', 'h_mavg_tranbal', 'i_mmax_tranbal', 'i_mavg_tranbal', 'j_mmax_tranbal', 'j_mavg_tranbal', 'k_mmax_tranbal', 'k_mavg_tranbal', 'm_mmax_tranbal', 'm_mavg_tranbal', 'n_mavg_trancount', 'n_mavg_tranbal', 'a_mdivision_tranbal', 'b_mdivision_tranbal', 'c_mdivision_tranbal', 'd_mdivision_tranbal', 'e_mdivision_tranbal', 'f_mdivision_tranbal', 'g_mdivision_tranbal', 'h_mdivision_tranbal', 'i_mdivision_tranbal', 'j_mdivision_tranbal', 'k_mdivision_tranbal', 'm_mdivision_tranbal', 'n_mdivision_tranbal', 'a_div_cus_asset_bal', 'b_div_cus_asset_bal', 'c_div_cus_asset_bal', 'd_div_cus_asset_bal', 'e_div_cus_asset_bal', 'f_div_cus_asset_bal', 'g_div_cus_asset_bal', 'h_div_cus_asset_bal', 'i_div_cus_asset_bal', 'j_div_cus_asset_bal', 'k_div_cus_asset_bal', 'm_div_cus_asset_bal', 'n_div_cus_asset_bal', 'pass_date1', 'pass_date1', 'pass_date2', 'pass_date', 'interval_mon_count1', 'interval_mon_count2', 'new12', 'new13', 'new14', 'new15', 'new16', 'new17', 'new18', 'new1', 'new2', 'new3', 'new4', 'new5', 'new6', 'new7', 'new8', 'new9', 'new10', 'new11', 'crcd1', 'crcd2', 'crcd3', 'crcd4', 'crcd5', 'interval_date1', 'cus_asset_bal1', 'cus_asset_bal2', 'cus_marr_stat1'
        ]
# feature=['crcd_is_overdue', 'cus_sex', 'crcd_is_pc', 'crcd_is_sup', 'cus_type', 'crcd_is_gec', 'crcd_bachange_cnt',
#              'cus_intvtime_trans', 'cus_edu', 'cus_marr_stat', 'cus_oact_pla', 'crcd_is_gc', 'crcd_bill_mcnt',
#              'cus_age', 'cus_mbank_lcnt', 'cus_active_ind', 'pass_date', 'interval_mon_count', 'interval_date_1stc', 'interval_date_1stc12', 'cus_hisloan_cnt', 'crcd_mavg_balind', 'cus_region_move_ind', 'cus_reloan_cap_ind', 'cus_asset_bal', 'crcd_max_quota', 'crcd_lmon_due_amount', 'crcd_due_amount', 'crcd_mavg_balance', 'crcd_points', 'a_mmax_tranbal', 'a_mavg_tranbal', 'b_mmax_tranbal', 'b_mavg_tranbal', 'c_mmax_tranbal', 'c_mavg_tranbal', 'd_mmax_tranbal', 'd_mavg_tranbal', 'e_mmax_tranbal', 'e_mavg_tranbal', 'f_mmax_tranbal', 'g_mmax_tranbal', 'g_mavg_tranbal', 'h_mmax_tranbal', 'h_mavg_tranbal', 'i_mmax_tranbal', 'i_mavg_tranbal', 'j_mmax_tranbal', 'j_mavg_tranbal', 'k_mmax_tranbal', 'k_mavg_tranbal', 'm_mmax_tranbal', 'm_mavg_tranbal', 'n_mavg_trancount', 'n_mavg_tranbal', 'a_mdivision_tranbal', 'b_mdivision_tranbal', 'd_mdivision_tranbal', 'e_mdivision_tranbal', 'g_mdivision_tranbal', 'h_mdivision_tranbal', 'i_mdivision_tranbal', 'j_mdivision_tranbal', 'k_mdivision_tranbal', 'm_mdivision_tranbal', 'n_mdivision_tranbal', 'crcd_due_div_crcd_lmon_due', 'a_div_cus_asset_bal', 'b_div_cus_asset_bal', 'c_div_cus_asset_bal', 'd_div_cus_asset_bal', 'e_div_cus_asset_bal', 'f_div_cus_asset_bal', 'g_div_cus_asset_bal', 'h_div_cus_asset_bal', 'i_div_cus_asset_bal', 'j_div_cus_asset_bal', 'k_div_cus_asset_bal', 'm_div_cus_asset_bal', 'n_div_cus_asset_bal']

cago_feature='0,1,2,3,4,5,6,7,8,9,10,11,12,13,14'
cago_feature=''
feature_train=feature+['target']
feature_test=feature
dataset_train=dataset_train[feature_train]
dataset_test=dataset_test[feature_test]

params={'reg_alpha': 1, 'reg_lambda': 0.8, 'subsample': 1, 'colsample_bytree': 1,
    'max_depth': 4, 'num_leaves': 18, 'min_data_in_leaf': 20, 
    'bagging_freq': 6, 'bagging_fraction': 0.65,
    'feature_fraction': 0.55, 'min_gain_to_split': 0.2, 
    'min_child_weight': 19.428902804238373, 'objective': 'binary',
    'learning_rate': 0.01,'boosting': 'gbdt', 'verbosity': -1, 'metric': 'auc', 
    'num_threads': 40,
    'is_unbalance': True, 'categorical_feature': cago_feature}
params={
    'num_leaves':12,
    'min_data_in_leaf':100,
    'objective':'binary',
    'max_depth':-1,
    'learning_rate':0.02,
    'boosting':'gbdt',
    'bagging_freq':8,
    'bagging_fraction':0.8,
    'feature_fraction':0.4,
    'bagging_seed':8,
    'reg_alpha':8,
    'reg_lambda':33,
    'random_state':42,
    'metric':'auc',
    'verbosity':-1,
    'subsample':1,
    'min_gain_to_split':0.01,
    'min_child_weight':10,
    'num_threads':-1,
    'is_unbalance': True
}
dataset_train_x,dataset_train_y=get_sample(dataset_train,target_0=1,target_1=1,seed=None)
X_train,X_test,Y_train,Y_test=train_test_split(dataset_train_x[feature],dataset_train_y,test_size=0.2,random_state=None)

lgb_train = lgb.Dataset(X_train,Y_train)
lgb_eval = lgb.Dataset(X_test,Y_test,reference=lgb_train)

model=lgb.train(params,lgb_train,num_boost_round=4000,valid_sets=lgb_eval,early_stopping_rounds=200)#,init_model=model)


[1]	valid_0's auc: 0.643251
Training until validation scores don't improve for 200 rounds
[2]	valid_0's auc: 0.660305
[3]	valid_0's auc: 0.662187
[4]	valid_0's auc: 0.663331
[5]	valid_0's auc: 0.666562
[6]	valid_0's auc: 0.66808
[7]	valid_0's auc: 0.668887
[8]	valid_0's auc: 0.670027
[9]	valid_0's auc: 0.67238
[10]	valid_0's auc: 0.674205
[11]	valid_0's auc: 0.675127
[12]	valid_0's auc: 0.675631
[13]	valid_0's auc: 0.676273
[14]	valid_0's auc: 0.676722
[15]	valid_0's auc: 0.677755
[16]	valid_0's auc: 0.67872
[17]	valid_0's auc: 0.678644
[18]	valid_0's auc: 0.678765
[19]	valid_0's auc: 0.678665
[20]	valid_0's auc: 0.678971
[21]	valid_0's auc: 0.679172
[22]	valid_0's auc: 0.679205
[23]	valid_0's auc: 0.679554
[24]	valid_0's auc: 0.680054
[25]	valid_0's auc: 0.680532
[26]	valid_0's auc: 0.680933
[27]	valid_0's auc: 0.681533
[28]	valid_0's auc: 0.682145
[29]	valid_0's auc: 0.682242
[30]	valid_0's auc: 0.682385
[31]	valid_0's auc: 0.683084
[32]	valid_0's auc: 0.682965
[33]	valid_0's auc: 0.